In [1]:
from amdex import *
import hashlib
import os
import json

# url = "https://amdex_server:8081/api/isJobAllowed/"
# token = "abc123"

# response = make_rest_call(url, token)
# if response:
#     print(response)
# else:
#     print("Error making REST API call")

In [2]:
# Example usage
name = "Predictive maintenance server"
password = "yes"
password_hash = hashlib.sha256(password.encode()).hexdigest().upper()

print("password_hash", password_hash)


password_hash 8A798890FE93817163B10B5F7BD2CA4D25D84C52739A645A889C173EEE7D9D3D


In [3]:
version = fetchApi("version")
print("version", version)

# Fetch authorization
auth_result = fetchAuth(name, password_hash)
print(auth_result)


# bearer_token = auth_result["bearer_token"]
# account_uuid = auth_result["account_uuid"]

# if(auth_result["status"] == "error"):

# print("Fetched authorization")
# print("Bearer token: " + bearer_token)  
# print("Account UUID: " + account_uuid)

bearer_token = ""
account_uuid = ""


version {'version': '0.0.2', 'api_version': '1', 'commit': 'ceca8d77704d76cbf83880e89a3d600c6c1090e6'}
fetchAuth - url:https://develop.amdex.dev/api/auth
fetchAuth - password_hash:8A798890FE93817163B10B5F7BD2CA4D25D84C52739A645A889C173EEE7D9D3D
fetchAuth - json:{"name": "Predictive maintenance server", "password_hash": "8A798890FE93817163B10B5F7BD2CA4D25D84C52739A645A889C173EEE7D9D3D"}
{'name': 'Predictive maintenance server', 'public_key': 'MIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEA2WgGfdnqUL0qwQ0sG3LVuAVX9JST4/W5yvAjnJlPEC33YVagYghQrMqhg/QCku2wSXCsztuwAaf9ugqZemheP8RfYeQkbm0d6wzXpOj6R7wBBV7j+aFAkbBlAKNKWyDAvn2HcS1jHNtcNfJQsaez6P5B1pC7OJdi+w1cHdgwLxIz0Kdm2nSYhUrnOZtDyl4hV4kwSFHiaFpIpihyTcbQhlnyNjE75auu6cL8G/wFBo/LPVNZmirWWa1Dvwfle20hc5AOsIFbMeHMnoVbsGFHBRO3eBMVdjGFrxv6ITW0juqpoHypjenVR4G3soklInr2Z4koHRBhM1yPoG3cpkuzMQIDAQAB', 'auditlog_write_access': True, 'auditlog_read_access_for_entries_by_account_uuids': ['6ae1ae4f-3c5c-4d55-8785-86076c6586f9', '886fc572-2522-4bf1-9399-3fca39fa9

In [4]:
# Fetch jobs
jobs_result = fetchJobs(account_uuid, bearer_token)
print("Jobs:", jobs_result)


Error during fetchJobs: 401 Client Error: Unauthorized for url: https://develop.amdex.dev/api/authenticated/jobs
Jobs: None


In [5]:


# Poll job status
job_uuid = "example_job_uuid"
status = pollJobStatus(account_uuid, job_uuid, bearer_token)
print("Job Status:", status)

# Fetch audit logs
auditlogs_result = fetchAuditlogs(account_uuid, job_uuid, bearer_token)
print("Audit Logs:", auditlogs_result)

# Fetch dataviews
dataviews_result = fetchDataviews(account_uuid, bearer_token)
print("Dataviews:", dataviews_result)

# Fetch API
api_function = "example"
api_result = fetchApi(api_function)
print("API Result:", api_result)    

Error during fetchJobs: 401 Client Error: Unauthorized for url: https://develop.amdex.dev/api/authenticated/jobs
Error during pollJobStatus: 'NoneType' object is not iterable
Job Status: None
Error during fetchAuditlogs: 401 Client Error: Unauthorized for url: https://develop.amdex.dev/api/authenticated/auditlogs/example_job_uuid
Audit Logs: None
Error during fetchDataviews: 401 Client Error: Unauthorized for url: https://develop.amdex.dev/api/authenticated/dataviews
Dataviews: None
Error during fetchApi: 404 Client Error: Not Found for url: https://develop.amdex.dev/api/example
API Result: None


In [6]:
# Fetch audit logs
auditlogs_result = fetchAuditlogs(account_uuid, job_uuid, bearer_token)
print("Audit Logs:", auditlogs_result)

# Fetch dataviews
dataviews_result = fetchDataviews(account_uuid, bearer_token)
print("Dataviews:", dataviews_result)

# Fetch API
api_function = "example"
api_result = fetchApi(api_function)
print("API Result:", api_result)    

Error during fetchAuditlogs: 401 Client Error: Unauthorized for url: https://develop.amdex.dev/api/authenticated/auditlogs/example_job_uuid
Audit Logs: None
Error during fetchDataviews: 401 Client Error: Unauthorized for url: https://develop.amdex.dev/api/authenticated/dataviews
Dataviews: None
Error during fetchApi: 404 Client Error: Not Found for url: https://develop.amdex.dev/api/example
API Result: None
